In [17]:
!pip install ultralytics

In [18]:
from ultralytics import YOLO
import numpy as np

In [19]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls -l '/content/dataset/data'

In [ ]:
import shutil
import os

!cp "/content/drive/MyDrive/data.zip" "/content/data.zip"

!unzip -q "/content/data.zip" -d "/content/dataset"


In [22]:
# val with baseline pretrained yolo11 model
model = YOLO('yolo11s.pt')

metrics = model.val(data='/content/dataset/data/bdd100k.yaml', save_json=True)

print(f"Mean Average Precision (mAP50-95): {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.mp:.4f}")
print(f"Recall: {metrics.box.mr:.4f}")

class_names = model.names
for i, score in enumerate(metrics.box.maps):
    print(f"Class {class_names[i]}: {score:.4f}")

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
YOLO11s summary (fused): 100 layers, 9,443,760 parameters, 0 gradients, 21.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1833.9±496.7 MB/s, size: 61.4 KB)
val: Scanning /content/dataset/data/labels/val... 10000 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 10000/10000 1.5Kit/s 6.8s
val: New cache created: /content/dataset/data/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 625/625 8.1it/s 1:17
                   all      10000     185578      0.139      0.106      0.109     0.0594
                person       3220      13265      0.612      0.444      0.467       0.23
               bicycle        515        649     0.0222     0.0185     0.0086    0.00265
                   car       9879     102540      0.693      0.509      0.583      0.348
            motorcycle       2689       4247          0          0

In [ ]:
!cp -r runs/detect/untrained_run '/content/drive/MyDrive/DS 340 Project/YOLO_Results/'

In [ ]:
# test training run w/ 15 epochs
model = YOLO('yolo11n.pt')
print("--- Starting Training ---")

drive_data_path = '/content/dataset/data/bdd100k.yaml'
results = model.train(
    data=drive_data_path,
    epochs=15,
    imgsz=512,
    batch=-1,
    name='bdd_colab_run',
    device='cuda',
    patience=5,
    amp=True,
    cache=False,
    save_period=5,
    exist_ok=True,
    max_det=50,
    workers = 16,
    mosaic = 0.5
)

metrics = model.val(
    data=drive_data_path,
    split='val',
    name='bdd_test_run',
    device='cuda',
    imgsz=512,
    conf=0.50,
    iou=0.7,
    save_json=True,
    save_txt=True,
    plots=True
)

In [ ]:
!cp -r runs/detect/bdd_colab_run '/content/drive/MyDrive/DS 340 Project/YOLO_Results/'
!cp -r runs/detect/bdd_test_run '/content/drive/MyDrive/DS 340 Project/YOLO_Results/'

In [ ]:
# optimized, 50 epochs, default data aug
model = YOLO('yolo11s.pt')
print("--- Starting Training ---")

drive_data_path = '/content/dataset/data/bdd100k.yaml'
results = model.train(
    data=drive_data_path,
    epochs=50,
    imgsz=640,
    batch=256,
    name='bdd_colab_run2',
    device='cuda',
    patience=10,
    amp=True,
    cache=True,
    save_period=5,
    exist_ok=True,
    max_det=50,
    workers = 16,
    lr0 = .04,
    warmup_epochs = 4
)

metrics = model.val(
    data=drive_data_path,
    split='val',
    name='bdd_test_run2',
    device='cuda',
    imgsz=640,
    conf=0.50,
    iou=0.7,
    save_json=True,
    save_txt=True,
    plots=True
)

In [ ]:
!cp -r runs/detect/bdd_colab_run2 '/content/drive/MyDrive/DS 340 Project/YOLO_Results/'
!cp -r runs/detect/bdd_test_run2 '/content/drive/MyDrive/DS 340 Project/YOLO_Results/'

In [ ]:
# added data aug, 50 epochs, lowered val conf (increase recall, decrease precision)
model = YOLO('yolo11s.pt')
print("--- Starting Training ---")

drive_data_path = '/content/dataset/data/bdd100k.yaml'
results = model.train(
    data=drive_data_path,
    epochs=50,
    imgsz=640,
    batch=256,
    name='bdd_colab_run3',
    device='cuda',
    patience=10,
    amp=True,
    cache=True,
    save_period=5,
    exist_ok=True,
    max_det=50,
    workers = 16,
    lr0 = .04,
    warmup_epochs = 4,

    #data augment, mosiac makes it remember specifc details
    mosaic=1.0,
    scale=0.5,
    fliplr=0.5,
    flipud=0.0,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,

    #new for 3
    erasing=0.4,
    mixup=0.1,
    copy_paste=0.1
)

metrics = model.val(
    data=drive_data_path,
    split='val',
    name='bdd_test_run3',
    device='cuda',
    imgsz=640,
    conf=0.35,
    iou=0.7,
    save_json=True,
    save_txt=True,
    plots=True
)

In [ ]:
!cp -r runs/detect/bdd_colab_run3 '/content/drive/MyDrive/DS 340 Project/YOLO_Results/'
!cp -r runs/detect/bdd_test_run3 '/content/drive/MyDrive/DS 340 Project/YOLO_Results/'

In [ ]:
# same settings as before, but trying out a more complex model, same data aug from 3 and uses the medium model
model = YOLO('yolo11m.pt')
print("--- Starting Training ---")

drive_data_path = '/content/dataset/data/bdd100k.yaml'
results = model.train(
    data=drive_data_path,
    epochs=50,
    imgsz=640,
    batch=128,
    name='bdd_colab_run4',
    device='cuda',
    patience=10,
    amp=True,
    cache=True,
    save_period=5,
    exist_ok=True,
    max_det=50,
    workers = 16,
    lr0 = .04,
    warmup_epochs = 4,
    mosaic=1.0,
    scale=0.5,
    fliplr=0.5,
    flipud=0.0,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    erasing=0.4,
    mixup=0.1,
    copy_paste=0.1
)

metrics = model.val(
    data=drive_data_path,
    split='val',
    name='bdd_test_run4',
    device='cuda',
    imgsz=640,
    conf=0.35,
    iou=0.7,
    save_json=True,
    save_txt=True,
    plots=True
)

In [ ]:
!cp -r runs/detect/bdd_colab_run4 '/content/drive/MyDrive/DS 340 Project/YOLO_Results/'
!cp -r runs/detect/bdd_test_run4 '/content/drive/MyDrive/DS 340 Project/YOLO_Results/'

In [ ]:
!zip -r /content/bdd_colab_run4.zip /content/runs/detect/bdd_colab_run4

from google.colab import files
files.download('/content/bdd_colab_run4.zip')

In [ ]:
# trying out medium model still but with different data augmentations
model = YOLO('yolo11m.pt')
print("--- Starting Training ---")

drive_data_path = '/content/dataset/data/bdd100k.yaml'
results = model.train(
    data=drive_data_path,
    epochs=50,
    imgsz=640,
    batch=128,
    name='bdd_colab_run5',
    device='cuda',
    patience=10,
    amp=True,
    cache=True,
    save_period=5,
    exist_ok=True,
    max_det=50,
    workers = 16,
    lr0 = .04,
    warmup_epochs = 4,
    mosaic=1.0,
    scale=0.5,
    fliplr=0.5,
    flipud=0.0,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    mixup = 0.2,
    degrees = 5.0,
    close_mosaic=1
)

metrics = model.val(
    data=drive_data_path,
    split='val',
    name='bdd_test_run5',
    device='cuda',
    imgsz=640,
    conf=0.35,
    iou=0.7,
    save_json=True,
    save_txt=True,
    plots=True
)

In [ ]:
!cp -r runs/detect/bdd_colab_run5 '/content/drive/MyDrive/DS 340 Project/YOLO_Results/'
!cp -r runs/detect/bdd_test_run5 '/content/drive/MyDrive/DS 340 Project/YOLO_Results/'

In [ ]:
!zip -r /content/bdd_colab_run5.zip /content/runs/detect/bdd_colab_run5

from google.colab import files
files.download('/content/bdd_colab_run5.zip')

In [ ]:
!cp -r runs/detect/bdd_test_run5 '/content/drive/MyDrive/DS 340 Project/YOLO_Results/'

In [39]:
from ultralytics import YOLO

model = YOLO('/content/drive/MyDrive/DS 340 Project/YOLO_Results/bdd_colab_run5/weights/best.pt')

results = model.predict(
    source='/content/IMG_1603.HEIC',
    save=True,
    show_labels=False,
    show_conf=False,
    conf=0.25,      # Confidence threshold (only show boxes > 25% sure)
    iou=0.45        # NMS threshold (remove overlapping boxes)
)


image 1/1 /content/IMG_1603.HEIC: 640x480 1 person, 8 cars, 2 traffic lights, 4 traffic signs, 12.8ms
Speed: 3.4ms preprocess, 12.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)
Results saved to /content/runs/detect/predict3


In [40]:
from ultralytics import YOLO

model = YOLO('/content/drive/MyDrive/DS 340 Project/YOLO_Results/bdd_colab_run5/weights/best.pt')

results = model.predict(
    source='/content/IMG_1604.HEIC',
    save=True,
    show_labels=False,
    show_conf=False,
    conf=0.25,      # Confidence threshold (only show boxes > 25% sure)
    iou=0.45        # NMS threshold (remove overlapping boxes)
)


image 1/1 /content/IMG_1604.HEIC: 480x640 4 persons, 3 cars, 1 truck, 3 traffic lights, 6 traffic signs, 80.4ms
Speed: 3.4ms preprocess, 80.4ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)
Results saved to /content/runs/detect/predict4


In [41]:
from ultralytics import YOLO

model = YOLO('/content/drive/MyDrive/DS 340 Project/YOLO_Results/bdd_colab_run5/weights/best.pt')

results = model.predict(
    source='/content/IMG_1605.HEIC',
    save=True,
    show_labels=False,
    show_conf=False,
    conf=0.25,      # Confidence threshold (only show boxes > 25% sure)
    iou=0.45        # NMS threshold (remove overlapping boxes)
)


image 1/1 /content/IMG_1605.HEIC: 480x640 7 persons, 12 cars, 2 traffic lights, 2 traffic signs, 12.9ms
Speed: 3.4ms preprocess, 12.9ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
Results saved to /content/runs/detect/predict5
